In [1]:
import numpy as np
import pandas as pd
from astropy.cosmology import LambdaCDM
import matplotlib.pyplot as plt
import treecorr
from astropy.io import fits
import sys
sys.path.append('/home/fcaporaso/modified_gravity/')
from lensing.funcs import lenscat_load

cosmology = LambdaCDM(H0=100.0, Om0=0.3089, Ode0=0.6911)


In [2]:
def d_com(z):
    global cosmology
    return cosmology.comoving_distance(z).value

In [3]:
def make_randoms(ra, dec, redshift,
                 size_random = 100):
    
    print('Making randoms...')
    np.random.seed(1)
    
    dec = np.deg2rad(dec)
    sindec_rand = np.random.uniform(np.sin(dec.min()), np.sin(dec.max()), size_random)
    dec_rand = np.arcsin(sindec_rand)*(180/np.pi)
    ra_rand  = np.random.uniform(ra.min(), ra.max(), size_random)

    y,xbins  = np.histogram(redshift, 25)
    x  = xbins[:-1]+0.5*np.diff(xbins)
    n = 3
    poly = np.polyfit(x,y,n)
    zr = np.random.uniform(redshift.min(),redshift.max(),size_random)
    poly_y = np.poly1d(poly)(zr)
    poly_y[poly_y<0] = 0.
    peso = poly_y/sum(poly_y)
    z_rand = np.random.choice(zr,size_random,replace=True,p=peso)

    print('Wii randoms!')
    return pd.DataFrame({'ra': ra_rand, 'dec': dec_rand, 'redshift':z_rand})


In [4]:
class Catalogos:
    def __init__(self, cat_config, lens_name, source_name):
        path = '/home/fcaporaso/cats/L768/'
        
        self.lenses = pd.DataFrame(
            lenscat_load(
                path+lens_name,
                *cat_config.values(),
                1,1
            )[0].T,
            columns=['rv',
                     'ra','dec','redshift',
                     'xv','yv','zv',
                     'rho1','rho2',
                     'logp','cmdist',
                     'flag']
        )
        
        self.sources = pd.read_parquet(path+source_name).sample(frac=0.5, random_state=1)
        self.sources.rename(
            columns={
                'ra_gal':'ra',
                'dec_gal':'dec',
                'true_redshift_gal':'redshift',
                'r_gal':'r_com'},
            inplace=True
        )
        query = f'redshift < {cat_config["z_max"]}+0.1 and redshift >= {cat_config["z_min"]}-0.1'
        self.sources.query(query,inplace=True)
        
        self.random_lenses = make_randoms(
            self.lenses.ra,
            self.lenses.dec,
            self.lenses.redshift,
            size_random=len(self.lenses)*2
        )

        self.random_sources = make_randoms(
            self.sources.ra,
            self.sources.dec,
            self.sources.r_com, 
            size_random=len(self.sources)*2
        )
        
        self.lenses['w'] = np.ones(len(self.lenses))
        self.sources['w'] = np.ones(len(self.sources))
        self.random_lenses['w'] = np.ones(len(self.random_lenses))
        self.random_sources['w'] = np.ones(len(self.random_sources))
        
        self.lenses['r_com'] = d_com(self.lenses.redshift)
        self.random_lenses['r_com'] = d_com(self.random_lenses.redshift)
        self.random_sources.rename(columns={'redshift':'r_com'}, inplace=True)


In [5]:
class VoidGalaxyCrossCorrelation:
    
    def __init__(self, config_treecorr):
        self.config : dict = config_treecorr
        
    def load_treecorrcatalogs(self, lenses, sources, random_lenses, random_sources):
        
        ## Voids
        self.dvcat = treecorr.Catalog(
            ra=lenses.ra,
            dec=lenses.dec,
            w=lenses.w,
            r=lenses.r_com,
            npatch=self.config['NPatches'],
            ra_units='deg',
            dec_units='deg',
        )

        ## Tracers (gx)
        self.dgcat = treecorr.Catalog(
            ra=sources.ra, 
            dec=sources.dec, 
            w = sources.w, 
            r=sources.r_com, 
            patch_centers= self.dvcat.patch_centers,
            ra_units='deg', dec_units='deg'
        )

        ## Random voids
        self.rvcat = treecorr.Catalog(
            ra=random_lenses.ra, 
            dec=random_lenses.dec, 
            w = random_lenses.w, 
            r=random_lenses.r_com, 
            patch_centers= self.dvcat.patch_centers,
            ra_units='deg', dec_units='deg'
        )

        ## Random tracers (gx)
        self.rgcat = treecorr.Catalog(
            ra=random_sources.ra, 
            dec=random_sources.dec, 
            w = random_sources.w, 
            r=random_sources.r_com, 
            patch_centers= self.dvcat.patch_centers,
            ra_units='deg', dec_units='deg'
        )

    def calculate_corr(self):
        
        DvDg = treecorr.NNCorrelation(
            nbins=self.config['nbins'], 
            min_sep=self.config['rmin'], 
            max_sep=self.config['rmax'], 
            bin_slop=self.config['slop'], brute = False, 
            verbose=0, var_method = 'jackknife',
            bin_type='Linear'
        )

        DvRg = treecorr.NNCorrelation(
            nbins=self.config['nbins'], 
            min_sep=self.config['rmin'], 
            max_sep=self.config['rmax'], 
            bin_slop=self.config['slop'], brute = False, 
            verbose=0, var_method = 'jackknife',
            bin_type='Linear'
        )

        RvDg = treecorr.NNCorrelation(
            nbins=self.config['nbins'], 
            min_sep=self.config['rmin'], 
            max_sep=self.config['rmax'], 
            bin_slop=self.config['slop'], brute = False, 
            verbose=0, var_method = 'jackknife',
            bin_type='Linear'
        )

        RvRg = treecorr.NNCorrelation(
            nbins=self.config['nbins'], 
            min_sep=self.config['rmin'], 
            max_sep=self.config['rmax'], 
            bin_slop=self.config['slop'], brute = False, 
            verbose=0, var_method = 'jackknife',
            bin_type='Linear'
        )
        
        DvDg.process(self.dvcat, self.dgcat, num_threads=self.config['ncores'])
        DvRg.process(self.dvcat, self.rgcat, num_threads=self.config['ncores'])
        RvDg.process(self.rvcat, self.dgcat, num_threads=self.config['ncores'])
        RvRg.process(self.rvcat, self.rgcat, num_threads=self.config['ncores'])

        self.r = DvDg.meanr
        self.xi, self.varxi = DvDg.calculateXi(dr=DvRg, rd=RvDg, rr=RvRg)
        self.cov = DvDg.cov
    
    def run(self, cats):
        self.load_treecorrcatalogs(
            cats.lenses,
            cats.sources,
            cats.random_lenses,
            cats.random_sources
        )
        self.calculate_corr()
        
    def plot_corr(self, label):

        fig = plt.figure(figsize=(8,6))
        ax = fig.add_axes([1,1,1,1])
        ax.errorbar(self.r, self.xi, self.varxi, fmt='.-', capsize=2, c='purple', label=label)
        ax.set_xlabel('meanr [Mpc/h]')
        ax.set_ylabel('$\\xi$')
        return ax
        # plt.title('fR')
        #plt.text(40,-0.6, f'$R_v \\in$ ({Rv_min},{Rv_max})')
        #plt.text(40,-0.65, f'$z \\in$ ({z_min},{z_max})')

## Catalogos a correlacionar

In [6]:
Rv_min = 15.0 # (0, 100)
Rv_max = 17.0 # (Rv_min+dR, 100)
z_min = 0.5 # (0.0, 0.6)
z_max = 0.6 # (z_min, 0.6)
rho1_min = -1.0 # en gral, no tocar
rho1_max = 0.0 # -0.8 es más correcto, pero da igual...
rho2_min = 0.0 # (-1,100)
rho2_max = 100.0 # (rho2_min, 100)
flag = 2.0 # 2 == voids muy confiables

In [7]:
cat_config = {
    'Rv_min':Rv_min,
    'Rv_max':Rv_max,
    'z_min':z_min,
    'z_max':z_max,
    'rho1_min':rho1_min,
    'rho1_max':rho1_max,
    'rho2_min':rho2_min,
    'rho2_max':rho2_max,
    'flag':flag,
}

In [8]:
lens_name = ['voids_LCDM_09.dat', 'voids_fR_09.dat']
source_name = ['l768_gr_galaxiesz00-07_bucket1of3_19814.parquet',
               'l768_mg_galaxiesz00-07_bucket1of3_19813.parquet']

In [9]:
cats_LCDM = Catalogos(cat_config, lens_name[0], source_name[0])
cats_fR = Catalogos(cat_config, lens_name[1], source_name[1])

Making randoms...
Wii randoms!
Making randoms...
Wii randoms!
Making randoms...
Wii randoms!
Making randoms...
Wii randoms!


### Histogramas para chequear randomness

#### LCDM

In [ ]:
plt.hist(cats_LCDM.lenses.rv,density=True,histtype='step')
plt.xlabel('$R_v$')
plt.show()

In [ ]:
plt.hist(cats_LCDM.lenses.redshift, density=True, histtype='step',label='lenses')
plt.hist(cats_LCDM.sources.redshift, density=True, histtype='step',label='sources')
plt.xlabel('redshift')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(3,2,
                      figsize=(6,9))

# axL,axS = ax[:,0], ax[:,1]

for j,name in enumerate(['ra','dec','r_com']):

    ax[j,0].hist(cats_LCDM.lenses[name], histtype='step', bins=50, density=True)
    ax[j,0].hist(cats_LCDM.random_lenses[name], histtype='step', bins=50, density=True)
    
    ax[j,1].hist(cats_LCDM.sources[name], histtype='step', bins=50, density=True)
    ax[j,1].hist(cats_LCDM.random_sources[name], histtype='step', bins=50, density=True)

# ax[-1,-1].legend()
plt.show()

#### fR

In [ ]:
plt.hist(cats_fR.sources.r_com, bins=50, histtype='step', density=True)
plt.hist(cats_fR.random_sources.r_com, bins=50, histtype='step', density=True)
plt.show()

In [ ]:
plt.hist(cats_fR.lenses.rv,density=True,histtype='step')
plt.xlabel('$R_v$')
plt.show()

In [ ]:
plt.hist(cats_fR.lenses.redshift, density=True, histtype='step',label='lenses')
plt.hist(cats_fR.sources.redshift, density=True, histtype='step',label='sources')
plt.xlabel('redshift')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(3,2,
                      figsize=(6,9))

# axL,axS = ax[:,0], ax[:,1]

for j,name in enumerate(['ra','dec','r_com']):

    ax[j,0].hist(cats_fR.lenses[name], histtype='step', bins=50, density=True)
    ax[j,0].hist(cats_fR.random_lenses[name], histtype='step', bins=50, density=True)
    
    ax[j,1].hist(cats_fR.sources[name], histtype='step', bins=50, density=True)
    ax[j,1].hist(cats_fR.random_sources[name], histtype='step', bins=50, density=True)

# ax[-1,-1].legend()
plt.show()

In [ ]:
plt.hist(cats_fR.sources.r_com, bins=50, histtype='step', density=True)
plt.hist(cats_fR.random_sources.r_com, bins=50, histtype='step', density=True)
plt.show()

## VGCF calculation

### LCDM

In [10]:
nbins = 25
rmin = 0.01*cats_LCDM.lenses.rv.mean()
rmax = 5.0*cats_LCDM.lenses.rv.mean()
ncores = 64

# parameters related with bins
config = {
    'nbins' : nbins, # number of radial bins
    'rmin' : rmin, # minimum value for rp (r in case of the quadrupole)
    'rmax' : rmax, # maximum value for rp (r in case of the quadrupole)
    # Related to JK patches
    'NPatches' : int(nbins**(3./2.)),
    # Other configuration parameters
    'ncores' : ncores, # Number of cores to run in parallel
    'slop' : 0., # Resolution for treecorr
    'box' : False, # Indicates if the data corresponds to a box, otherwise it will assume a lightcone
} 

print('Number of patches',config['NPatches'])
print(f'r = ({np.round(config["rmin"],2)}, {np.round(config["rmax"],2)})')

Number of patches 125
r = (0.16, 79.38)


In [11]:
vgcf_LCDM = VoidGalaxyCrossCorrelation(config)

In [ ]:
vgcf_LCDM.run(cats_LCDM)

The following patch numbers have no objects: {76}
This may be a problem depending on your use case.
The following patch numbers have no objects: {76}
This may be a problem depending on your use case.


In [ ]:
ax = vgcf_LCDM.plot_corr()
ax.axhline(0,ls=':',c='gray')

### fR

In [ ]:
nbins = 25
rmin = 0.01*cats_fR.lenses.rv.mean()
rmax = 5.0*cats_fR.lenses.rv.mean()
ncores = 64

# parameters related with bins
config = {
    'nbins' : nbins, # number of radial bins
    'rmin' : rmin, # minimum value for rp (r in case of the quadrupole)
    'rmax' : rmax, # maximum value for rp (r in case of the quadrupole)
    # Related to JK patches
    'NPatches' : int(nbins**(3./2.)),
    # Other configuration parameters
    'ncores' : ncores, # Number of cores to run in parallel
    'slop' : 0., # Resolution for treecorr
    'box' : False, # Indicates if the data corresponds to a box, otherwise it will assume a lightcone
} 

print('Number of patches',config['NPatches'])
print(f'r = ({np.round(config["rmin"],2)}, {np.round(config["rmax"],2)})')

In [ ]:
vgcf_fR = VoidGalaxyCrossCorrelation(config)

In [ ]:
vgcf_fR.run(cats_fR)

In [ ]:
ax = vgcf_fR.plot_corr()
ax.axhline(0,ls=':',c='gray')

## Plots

In [ ]:
ax = vgcf_LCDM.plot_corr('LCDM')
ax.errorbar(vgcf_fR.r, vgcf_fR.xi, vgcf_fR.varxi, c='tomato', capsize=2, fmt='.-', label='fR')
ax.axhline(0, ls=':', c='gray')
ax.axvline(cats_LCDM.lenses.rv.mean(), ls=':', c='purple')
ax.axvline(cats_fR.lenses.rv.mean(), ls=':', c='tomato')
plt.legend()

In [ ]:
def err_frac2(a, b, e_a, e_b):
    """
    error de D = a/b -1 
    """
    # e_a2 = np.diag(cova)
    # e_b2 = np.diag(covb)
    # np.cov(cova,covb)
    
    return e_a/np.abs(b) + e_b/np.abs(a)

In [ ]:
fig,(ax1, ax2) = plt.subplots(2,1,
                            figsize=(8,10),
                            sharex=True,
                            gridspec_kw={'height_ratios': [2,1]})

ax1.errorbar(LCDM_R15_17_z01_02_R['r'],LCDM_R15_17_z01_02_R['xi'],LCDM_R15_17_z01_02_R['varxi'],
            label='LCDM halos', fmt='.-', capsize=2, c='b')
ax1.errorbar(LCDM_R15_17_z01_02_R_gx['r'],LCDM_R15_17_z01_02_R_gx['xi'],LCDM_R15_17_z01_02_R_gx['varxi'],
            label='LCDM gx', fmt='.-', capsize=2, c='cyan')

ax1.errorbar(fR_R15_17_z01_02_R['r'],fR_R15_17_z01_02_R['xi'],fR_R15_17_z01_02_R['varxi'],
            label='fR halos', fmt='.-', capsize=2, c='r')
ax1.errorbar(fR_R15_17_z01_02_R_gx['r'],fR_R15_17_z01_02_R_gx['xi'],fR_R15_17_z01_02_R_gx['varxi'],
            label='fR gx', fmt='.-', capsize=2, c='orange')
ax2.set_xlabel('meanr [Mpc/h]')
ax1.set_ylabel('$\\xi$')
ax1.legend()

ax2.errorbar(LCDM_R15_17_z01_02_R['r'], LCDM_R15_17_z01_02_R['xi']/LCDM_R15_17_z01_02_R_gx['xi'] - 1, 
            err_frac2(LCDM_R15_17_z01_02_R['xi'], LCDM_R15_17_z01_02_R_gx['xi'], LCDM_R15_17_z01_02_R['varxi'],
                     LCDM_R15_17_z01_02_R_gx['varxi']),
            c='b',fmt='.-',capsize=2, label='LCDM dif')
ax2.errorbar(fR_R15_17_z01_02_R['r'], fR_R15_17_z01_02_R['xi']/fR_R15_17_z01_02_R_gx['xi'] - 1, 
            err_frac2(fR_R15_17_z01_02_R['xi'], fR_R15_17_z01_02_R_gx['xi'], fR_R15_17_z01_02_R['varxi'],
                     fR_R15_17_z01_02_R_gx['varxi']),
            c='r',fmt='.-',capsize=2, label='fR dif')

ax2.errorbar(LCDM_R15_17_z01_02_R_gx['r'], LCDM_R15_17_z01_02_R_gx['xi']/fR_R15_17_z01_02_R_gx['xi'] - 1, 
            err_frac2(LCDM_R15_17_z01_02_R_gx['xi'], fR_R15_17_z01_02_R_gx['xi'], LCDM_R15_17_z01_02_R_gx['varxi'],
                     fR_R15_17_z01_02_R_gx['varxi']),
            c='k',fmt='.-',capsize=2, label='LCDM/fR gx')


ax2.axhline(0,ls=':',c='k',alpha=0.5)
ax2.axhline(0.1,ls='--',c='k',alpha=0.5)
ax2.axhline(-0.1,ls='--',c='k',alpha=0.5)

ax2.set_ylabel('DIFF')
ax2.legend()

plt.subplots_adjust(wspace=0.01, hspace=0.05)